$\textbf{IMPORTS}$

In [1]:
import requests
import json
import time
import pprint
import pandas as pd
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

$\textbf{LOAD ALL PAGES}$, BOLIGA.DK

In [2]:
#1.Load pages boliga.dk
links = []
for page in range(1,149):
    url_boliga = f'https://www.boliga.dk/resultat?propertyType=3&page={page}&sort=daysForSale-a'
    links.append(url_boliga)

$\textbf{SCRAPE DATA FROM EACH PAGE}$, BOLIGA.DK

In [3]:
#Combinale all code above in a loop to scrape all pages
add = []
p = []
zipc = []
m_2 = []
rooms_ = []

for i in links:
    response = requests.get(i, headers={'name':'Sevda Öztürk','email':'txg194b@alumni.ku.dk'})
    soup = BeautifulSoup(response.content,"xml")
    
    #append address to list
    list_address = soup.find_all("div" ,class_ = 'w-100 mt-1 listing-address font-weight-bold text-nowrap text-truncate')
    address = [col.text.split(",",1)[0] for col in list_address]
    add.append(address)
    #convert list of lists to ONE LIST
    address_final = []
    for sublist in add:
        for item in sublist:
            address_final.append(item)
    
    #append price to list
    list_price = soup.find_all("div" , class_ ="price m-0 d-flex align-items-center")
    price = [col.text for col in list_price]
    p.append(price)
    #convert list of lists to ONE LIST
    price_final = []
    for sublist in p:
        for item in sublist:
            price_final.append(item)
    
    #append zipcode to list
    list_zipcode = soup.find_all("div" ,class_ = "zip-color text-nowrap text-truncate")
    zipcode = [col.text for col in list_zipcode]
    zipc.append(zipcode)
    #convert list of lists to ONE LIST
    zipcode_final = []
    for sublist in zipc:
        for item in sublist:
            zipcode_final.append(item)
    
    #append m2 to list
    list_m2 = soup.find_all("span" ,class_ = "text-nowrap")
    m2 = [col.text.split() for col in list_m2]
    #the variable m2 is a list of lists. The following code keeps the first element of every sublist to get rid of the string "m2"
    m2 = [element[0] for element in m2]
    del m2[0] #delete first element in m2 in order to get all m2 in next code
    m2 = m2[::6] #we now keep every 6th element, which is the size of every address
    m_2.append(m2)
    #convert list of lists to ONE LIST
    m2_final = []
    for sublist in m_2:
        for item in sublist:
            m2_final.append(item)
    
    #append room to list
    rooms = [col.text.split() for col in list_m2]
    rooms = [elements[1:] for elements in rooms]
    rooms = rooms[::6] #we now keep every 6th element, which is no. of rooms
    rooms_.append(rooms)
    #convert list of lists to ONE LIST
    rooms_fin = []
    for sublist in rooms_:
        for item in sublist:
            rooms_fin.append(item)
    rooms_final = []
    for sublist in rooms_fin:
        for item in sublist:
            rooms_final.append(item)
    #replace "værelse" with "1"
    for i in range(len(rooms_final)):
        if rooms_final[i] == "værelse":
            rooms_final[i] = "1"
    time.sleep(0.5) #Sleep for 0.5 seconds

$\textbf{COMBINE ALL CODE ABOVE INTO ONE DATAFRAME}$

In [153]:
#1. Combine all scraped lists in one dataframe
df_addzip = {"price": price_final, "address": address_final, "zipcode": zipcode_final, "m2": m2_final, "rooms": rooms_final}
df_data_ = pd.DataFrame(df_addzip)
df_data

,price,address,zipcode,m2,rooms
0,545.000kr.,Grønningen 29R,9800 Hjørring,46,2
1,550.000kr.,Frederikssundsvej 346,2700 Brønshøj,35,1
2,595.000kr.,Grøfthøjparken 151,8260 Viby J,33,1
3,995.000kr.,Stranden 70,9560 Hadsund,66,4
4,1.095.000kr.,Odinsvej 10,4700 Næstved,72,2
...,...,...,...,...,...
7249,600.000kr.,Chr. Møllers Vej 6D,9990 Skagen,72,3
7250,390.000kr.,Ringgade 82,7600 Struer,66,2
7251,595.000kr.,Tranevej 108,9990 Skagen,74,3
7252,345.000kr.,Fjelstervangvej 26A,6933 Kibæk,82,4


$\textbf{GET  DISTANCES}, \ dataforsyningen.dk, \ hvorlangterder.dk$

In [78]:
#1. Load pages
Address_zipcode = df_data["address"] + " " + df_data["zipcode"]
add_zip = []
for i in Address_zipcode:
    response_dist = requests.get(f"https://api.dataforsyningen.dk/adresser?q={i}&struktur=mini", headers={'name':'Sevda Öztürk','email':'txg194@alumni.ku.dk'})
    res_j = response_dist.json()
    add_zip.append(res_j)

Before continuing, locate missing values due to typing error in column address in df_data

In [154]:
#2. Locate missing values
df_add_zip = pd.DataFrame(add_zip)
index = df_add_zip[df_add_zip[0].isnull()][0]

#3. Store index numbers in order to del the corresponding indexes in df_data
index_list = list(index.index.values)

#4. Delete corresponding rows in df_data
for i in index_list:
    df_add_zip.drop(i, inplace=True)

#5. reset_index keeps failing, so here comes another index-method    
df_add_zip["new_index"] = range(len(df_add_zip))
df_add_zip.set_index("new_index", inplace=True)

#vi mister 7254 - 7114 observationer efter fjernelse af missing values!!

Drop missing values in df_data

In [162]:
#6. Delete corresponding rows in df_data
for i in index_list:
    df_data.drop(i, inplace=True)

#reset_index keeps failing, so here comes another index-method    
df_data["new_index"] = range(len(df_data))
df_data.set_index("new_index", inplace=True)

#vi mister 7254 - 7114 observationer efter fjernelse af missing values!!

In [165]:
#7. Get longitude, x, and latitudes, y from address i
x = []
y = []

for i in range(len(df_add_zip)):
    xx = df_add_zip[0][i]["x"]
    x.append(xx)

for i in range(len(df_add_zip)):
    yy = df_add_zip[0][i]["y"]
    y.append(yy)

In [168]:
#8. Add coordinates to df_data
df_data["x"] = x
df_data["y"] = y

In [169]:
#3 Get distance to nearest daycare
xy = df_data["y"].apply(str) +","+ df_data["x"].apply(str)
distance = []

for i in xy:
    afstand_daycare_json = requests.get(f'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInJlZiI6ICIyIiwgInBhciI6ICIiLCAicHJpdnMiOiAicjFaMHIwRjBrNkJ0V3FRY09VeWtCL3k2U1VwSnYyUWJneVhldHE0SFk0WE8vM1lyVzArSzl3PT0ifQ.2HJhvgEQPhJZuxuNZcR2y5jhmCn1IQpvqgr337BOI3e1ZzW1YmDVJDfELUKBTSXKNKlhq+1h+B5rO6rpLunNhw&fromlatlng={i}&poitypes=daycare&decodepolyline')
    afstand_daycare_json.json()
    key = afstand_daycare_json.json()['daycare']
    key = key['routedmeters']
    distance.append(key)

$\textbf{FINAL DATASET}$

In [170]:
#4 Add coordinates of addresses in dataframe
df_data["distance"] = distance
df_data

,price,address,zipcode,m2,rooms,x,y,distance
new_index,,,,,,,,
0,545.000kr.,Grønningen 29R,9800 Hjørring,46,2,10.007832,57.450271,695.26
1,550.000kr.,Frederikssundsvej 346,2700 Brønshøj,35,1,12.469893,55.712981,532.94
2,595.000kr.,Grøfthøjparken 151,8260 Viby J,33,1,10.136680,56.123838,229.74
3,995.000kr.,Stranden 70,9560 Hadsund,66,4,10.125431,56.718393,1405.77
4,1.095.000kr.,Odinsvej 10,4700 Næstved,72,2,11.770631,55.226144,1075.44
...,...,...,...,...,...,...,...,...
7109,600.000kr.,Chr. Møllers Vej 6D,9990 Skagen,72,3,10.521159,57.728456,2554.65
7110,390.000kr.,Ringgade 82,7600 Struer,66,2,8.598062,56.486802,220.35
7111,595.000kr.,Tranevej 108,9990 Skagen,74,3,10.465412,57.660428,8828.63
